In [49]:
import networkx as nx
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [50]:
def Create_channel(N,Channel_upperbound,option = "general",initial_length = 1):
    '''
    options:
        general:
            Random wire length from 0 to N virtual machines
        same, initial_length:
            Same wire length from 0 to N virtual machines. 
            If no specific details the progra will assume all items are 1
            The length cannot be 0 or the network will not take it as a normal edge.
    '''
    Channel_link = np.random.randint(Channel_upperbound,size=(N+1, N+1),) +1
    Channel_link = Channel_link - np.diag(np.diag(Channel_link)) + self_looped*np.eye(N+1,N+1)
    if option == "general":
        Channel_link[:,0] = 0
    elif option == 'same':
        Channel_link[0,:] = initial_length
        Channel_link[:,0] = 0
    else:
        print("Oops!  That was no valid option.  Try again...")
    
    return Channel_link

In [38]:
def Data_Queue(I,J,Data_upperbound):
    Data_Queue = np.random.randint(Data_upperbound,size=(I, J))+1
    return Data_Queue


In [39]:
def Data_Queue_type(I,J,K):
    Data_type_Queue = np.array([np.array(np.random.choice(K, J, replace = False)) for i in range(I)], dtype="object")+1
    return Data_type_Queue


In [40]:
def VM_Data_type_gen(I,K,N):
    flag = True
    while flag:
        type_selected = np.random.randint(1,K)
        VM_Data_type = np.sort(np.array([np.array(np.random.choice(K, type_selected, replace = False)) for i in range(N)])) + 1
        uniques = np.unique(VM_Data_type)
        flag = (len(uniques)<K)
    return VM_Data_type

In [41]:
def VM_Data_processing_speed(Processing_upperbound, VM_Data_type, K):
    N = len(VM_Data_type)
    VM_Data_speed_preset = np.array([np.array(np.random.choice(Processing_upperbound, K, replace = False)) for i in range(N)]) + 1
    VM_Data_speed_bolean = np.zeros((N,K))
    for row in range(N):
        VM_Data_speed_bolean[row,VM_Data_type[row,:]-1] = 1

    VM_Data_speed = VM_Data_speed_bolean * VM_Data_speed_preset
    return VM_Data_speed

In [42]:
def VM_Data_processing_time(Data_Queue,Data_type_Queue , VM_Data_speed):
    #
    I,J = np.shape(Data_Queue)
    N = len(VM_Data_speed)
    
    #
    General_Processing_time = np.ceil(np.kron(Data_Queue,1./VM_Data_speed))
    Processing_time_alt=np.ceil(np.reshape(General_Processing_time,(I,N,J,K)))
    Processing_time_final = 1./np.zeros((I,J,N))
    
    for i in range(I):
        for j in range(J):
            function_type = Data_type_Queue[i,j]-1

            Processing_time_final[i,j] = Processing_time_alt[i,:,j,function_type]
    
    return Processing_time_final

In [43]:
def Channel_Data_transmission_time(Data_Queue , Channel_link):
    #N' = N+1 contain initial node
    I,J = np.shape(Data_Queue)
    N_prime = len(Channel_link)
    
    Transmission_time = np.ceil(np.kron(Data_Queue,1./Channel_link))
    Transmission_time = np.reshape(Transmission_time,(I,N_prime,J,N_prime))
    
    return Transmission_time

In [44]:
def Draw_network(Channel_link):
    G = nx.from_numpy_matrix(Channel_link,create_using=nx.DiGraph(directed=True))
    edge_label = {}
    for j, edge in enumerate(G.edges()):
        #edge_label.update({(edge[0],edge[1]): "<"+ str(Channel_link[edge[0]+1,edge[1]+1])+ "|" +str(Channel_link_[edge[1]+1,edge[0]+1])+">" })
        #print(edge)
        if edge[1] > edge[0]:
            edge_label.update({(edge[0],edge[1]): "<" + str(int(Channel_link[edge[0],edge[1]])) + "|" })
        else:
            edge_label.update({(edge[0],edge[1]): "|" + str(int(Channel_link[edge[0],edge[1]])) + ">" })    
        pos = nx.circular_layout(G)
    #graph = nx.draw(G,pos,with_labels = True, connectionstyle='arc3, rad = 0.1')
    graph = nx.draw(G,pos,with_labels = True)
    graph = nx.draw_networkx_edge_labels(G,pos,edge_labels = edge_label,label_pos = 0.8)

In [45]:
def show_graph(matrix):
    matfig = plt.figure(figsize=(9,9)) 
    plt.matshow(matrix,fignum=matfig.number)
    plt.colorbar()
    plt.show()    

In [55]:
def show_relation(Final_map):
    #matfig = plt.figure(figsize=(Final_map.shape[0],Final_map.shape[1]))
    matfig = plt.figure(figsize=(9,9))
    plt.matshow(Final_map,cmap=plt.cm.binary,fignum=matfig.number)
    ax = plt.gca()
    #plt.matshow(Final_map,cmap=plt.cm.binary)
    plt.ylabel('Source node index')
    plt.xlabel('To node index')
    plt.title ("Direction map visualization")

    # Major ticks
    ax.set_xticks(np.arange(0, Final_map.shape[0], step=1))
    ax.set_yticks(np.arange(0, Final_map.shape[0], step=1))

    # Labels for major ticks
    ax.set_xticklabels(np.arange(0, Final_map.shape[0], step=1))
    ax.set_yticklabels(np.arange(0, Final_map.shape[0], step=1))

    # Minor ticks
    ax.set_xticks(np.arange(0.5, Final_map.shape[0]+0.5, step=1), minor=True)
    ax.set_yticks(np.arange(0.5, Final_map.shape[0]+0.5, step=1), minor=True)

    # Gridlines based on minor ticks
    ax.grid(which='minor', color='r', linestyle='-', linewidth=1)

    plt.show()